<a href="https://colab.research.google.com/github/zohour-diaa-eldin/Insurance-Classification/blob/main/Insurance_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e7:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73291%2F8930475%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240826%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240826T132237Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D584f450852311e10adc7cc757d27c7543f0d38ffcae4bc16fd0da8998e2d32d2c0c93a2063d29f9aeecee2bb9dc18433d777a4a0cb96e82ff2a500c7c0601ec92a62d23075aa9d8945146b2884c8040f159f3e2f319981d6202982626395acb4e2b2551b98c8799ea795ac896c6a140ef15dc16da70a3bce772073110cc5bedbe77ea2428818a218b781431da6e24ef7af6c854310803885df9ec1473eb88909d4988182cd8c5c33dd2ccf0688e12632375d65224e9ff4af997aea2591a679e0cdb777eb57a95b088b8539c12cab7f65759b5002b68e551a91dfabaf80491014e62aedd9adf6741b3b864455fa9498511355f68e4f86b50d70179809043f1f91'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 259402906 bytes downloaded
Downloaded and uncompressed: playground-series-s4e7
Data source import complete.


## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedShuffleSplit

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Load Data


In [3]:
df_train=pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")

In [4]:
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
df_train.columns

Index(['id', 'Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response'],
      dtype='object')

## Preprocess Data

In [6]:
def map_cat(df):
    df['Vehicle_Damage'] = df['Vehicle_Damage'].map({'Yes': 1, 'No': 0})
    df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
    df['Vehicle_Age'] = df['Vehicle_Age'].map({'< 1 Year': 1, '1-2 Year': 2, '> 2 Years': 3})
    return df

def add_feat(df):
    df['Age_Vehicle_Age'] = df['Age'] * df['Vehicle_Age']
    df['Age_Annual_Premium'] = df['Age'] * df['Annual_Premium']
    df['Vehicle_Damage_Annual_Premium'] = df['Vehicle_Damage'] * df['Annual_Premium']
    return df


In [7]:
df_train = map_cat(df_train)
df_test = map_cat(df_test)

df_train = add_feat(df_train)
df_test = add_feat(df_test)


## Prepare Data for Training

In [8]:
X = df_train.drop(['id', 'Response', 'Driving_License', 'Gender'], axis=1)
y = df_train['Response']


In [9]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.17, random_state=42)
for train_index, val_index in sss.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]


## Train the Model

In [10]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'device_type': 'cpu',
    'boosting_type': 'gbdt',
    'num_leaves': 864,
    'learning_rate': 0.075,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'lambda_l1': 0.15,
    'lambda_l2': 0.15,
    'max_depth': 39,
    'max_bin': 69754,
    'n_estimators': 4342
}

train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data, free_raw_data=False)

bst = lgb.train(
    params,
    train_data,
    num_boost_round=1500,
    valid_sets=[val_data],
    callbacks=[lgb.early_stopping(stopping_rounds=37), lgb.log_evaluation(50)]
)


[LightGBM] [Info] Number of positive: 1174499, number of negative: 8374483
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.005128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 101269
[LightGBM] [Info] Number of data points in the train set: 9548982, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964348
[LightGBM] [Info] Start training from score -1.964348
Training until validation scores don't improve for 37 rounds
[50]	valid_0's auc: 0.881442
[100]	valid_0's auc: 0.885247
[150]	valid_0's auc: 0.886955
[200]	valid_0's auc: 0.887739
[250]	valid_0's auc: 0.888353
[300]	valid_0's auc: 0.888509
[350]	valid_0's auc: 0.888612
[400]	valid_0's auc: 0.888968
[450]	valid_0's auc: 0.88912
[500]	valid_0's auc: 0.88914
[550]	valid_0's auc: 0.889252
[600]	valid_0's auc: 0.88927
[650]	valid_0's auc: 0.88935
Early stopping, best iteration is:
[657]	valid_0's a

## Make Predictions

In [16]:
#df_test=df_test.drop(columns=['id','Driving_License','Gender'],axis=1)
y_test=pd.read_csv("/kaggle/input/playground-series-s4e7/sample_submission.csv")
y_pred = bst.predict(df_test)

In [17]:


submission = pd.DataFrame({'id': y_test['id'], 'Response': y_pred})

submission.head()


,id,Response
0,11504798,0.005506
1,11504799,0.597382
2,11504800,0.241807
3,11504801,0.000119
4,11504802,0.109866


In [18]:
submission.to_csv('submission.csv', index=False)

In [19]:

from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>